<a href="https://colab.research.google.com/github/Sivasurya-J/DataScienceAcademicProjects/blob/main/IMDB_Movie_reviews_Sentiment_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import imdb

In [2]:
number_of_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

17464789/17464789 [==============================] - 1s 0us/step


**Data Exploration**

In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000,), (25000,), (25000,), (25000,))

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
X_train[123]

[1, 307, 5, 1301, 20, 1026, 2511, 87, 2775, 52, 116, 5, 31, 7, 4, 91, 1220, 102, 13, 28, 110, 11, 6, 137, 13, 115, 219, 141, 35, 221, 956, 54, 13, 16, 11, 2714, 61, 322, 423, 12, 38, 76, 59, 1803, 72, 8, 2, 23, 5, 967, 12, 38, 85, 62, 358, 99]

**Decoding a Movie Review**

In [6]:
word_to_index = imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [7]:
word_to_index['great']

84

In [8]:
index_to_word = {index: word for (word, index) in word_to_index.items()}

In [9]:
[index_to_word[i] for i in range(1, 51)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more']

In [10]:
' '.join([index_to_word.get(i - 3, '?') for i in X_train[123]])

'? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too'

In [11]:
y_train[123]

1

***Data Preparation***

In [12]:
words_per_review = 200

from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, maxlen=words_per_review)

In [13]:
X_train.shape

(25000, 200)

In [14]:
X_test = pad_sequences(X_test, maxlen=words_per_review)
X_test.shape

(25000, 200)

In [15]:
from sklearn.model_selection import train_test_split
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=11, test_size=0.20)

In [16]:
X_test.shape, X_val.shape

((20000, 200), (5000, 200))

**Creating Neural Network**

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Embedding

rnn=Sequential()
rnn.add(Embedding(input_dim=number_of_words, output_dim=128, input_length=words_per_review))
# adding LSTM layer
rnn.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
rnn.add(Dense(units=1, activation='sigmoid'))

In [18]:
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


**Training and Evaluating Model**

In [20]:
rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
782/782 [==============================] - 556s 707ms/step - loss: 0.4313 - accuracy: 0.7997 - val_loss: 0.3405 - val_accuracy: 0.8565
Epoch 2/10
782/782 [==============================] - 582s 744ms/step - loss: 0.2710 - accuracy: 0.8921 - val_loss: 0.3393 - val_accuracy: 0.8529
Epoch 3/10
782/782 [==============================] - 562s 719ms/step - loss: 0.1993 - accuracy: 0.9220 - val_loss: 0.3778 - val_accuracy: 0.8616
Epoch 4/10
782/782 [==============================] - 561s 717ms/step - loss: 0.1530 - accuracy: 0.9437 - val_loss: 0.4504 - val_accuracy: 0.8587
Epoch 5/10
782/782 [==============================] - 561s 717ms/step - loss: 0.1782 - accuracy: 0.9352 - val_loss: 0.4328 - val_accuracy: 0.8526
Epoch 6/10
782/782 [==============================] - 558s 714ms/step - loss: 0.1098 - accuracy: 0.9599 - val_loss: 0.4510 - val_accuracy: 0.8499
Epoch 7/10
782/782 [==============================] - 565s 723ms/step - loss: 0.0744 - accuracy: 0.9740 - val_loss: 0.5095 -

In [21]:
results = rnn.evaluate(X_test, y_test)

625/625 [==============================] - 51s 81ms/step - loss: 0.6186 - accuracy: 0.8399
